In [1]:
import scipy
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import MeanShift, estimate_bandwidth

import pandas as pd

from scipy import stats
from scipy.stats import beta
from math import sin
from random import randint

import matplotlib.pyplot as plt
import itertools as it

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)

import collections

def recursively_default_dict():
        return collections.defaultdict(recursively_default_dict)



## Coalescence algorithms.

Calculating the probability of sample configuration using population genetics models of mutation. 

    I. Infinite alleles - Ewens, 1972.
        i. recursive
        ii. exact.
       
    II. Infinite sites. 
            Griffiths, Ethier and Tavaré, 1987-1995
            Faisal, Futschik and Vogl (2010)

- Following Hein, Schierup and Wiuf, 20015. Chapter II. 

### References

- Ewens, W. J. (1972). The sampling theory of selectively neutral alleles. Theoretical population biology, 3(1), 87-112.

- Tavaré, S., Balding, D. J., Griffiths, R. C., & Donnelly, P. (1997). Inferring coalescence times from DNA sequence data. Genetics, 145(2), 505-518.

- Hein, J., Schierup, M., & Wiuf, C. (2004). Gene genealogies, variation and evolution: a primer in coalescent theory. Oxford University Press, USA.

- Faisal, M., Futschik, A., & Vogl, C. (2015). Exact Likelihood Calculation under the Infinite Sites Model. Computation, 3(4), 701-713.



## Presentation

### co-factors

#### mutation & coalescence

Probability that a either mutation or a coalescent event occurs first when considered backwards in generations. Each modelled after an exponential distribution of intensity:

   - a binomial coefficient, the number of combinations of 2 genes among _k_ possible genes, for coalescence;
    
   - _Theta_, or **4Nu**, the scaled mutation rate, for mutation.


The formulas are derived as the calculation of the minimum of these two quantities, an exponential of intensity equal to the sum of that of its components:

   - _min {Exp(a), Exp(b)} = Exp(a + b)_


In [2]:
def prob_coal(theta,nsamp):
    
    p= (nsamp - 1) / (nsamp - 1 + theta)
    
    return p

def prob_mut(theta,nsamp):
    
    p= theta / (nsamp - 1 + theta)
    
    return p


#### config.

Sample configuration. 

Under the infinite alleles model (Ewens 1972), it is assumed that no spatial or quantitaive information is known about alleles. We know only whether two alleles are different or identical. As a consequence, haplotype data sets can be treated as vectors of length _j_, where each element represents the number of allele classes possessing _j_ members.

As you can imagine, this makes considering the disapearance of alleles, through coalescence of identical forms or disapearnce of singletons through mutation, rather more simple. 

_Yet not so simple that to reproduce the algorithm is straighforward (see below)._

The function `get_config` draws the sample configuration of a numpy array. It considers rows as haplotypes, columns as loci.

In [3]:

def get_config(dataw,nsamp):
    hap_str= [''.join([str(c) for c in x]).strip() for x in dataw]
    hap_str= {z:[x for x in range(nsamp) if hap_str[x] == z] for z in set(hap_str)}
    
    class_len= np.array([len(hap_str[z]) for z in hap_str.keys()])
    
    config= [sum(class_len == x) for x in range(1,nsamp + 1)]
    return config


## Infinite alleles

_Ewens 1972_

    i. Recursion.

Recursion equation of probability of sample configuration as described by Ewens. Equation `2.13` of GGVE (see Index).


In [4]:

def Ewens_recurs(config_vec,theta,prob_array,Pin,prob_bound = 1):
    n_samp= sum([(x + 1) * config_vec[x] for x in range(len(config_vec))])
    
    if config_vec == [1]:
        ## boundary
        
        prob_left= Pin * prob_bound
        
        prob_array.append(prob_left)
        
        return prob_array
    
    if config_vec[0] > 0:
        ## mutation
        prob_left= prob_mut(theta,n_samp)
        
        new_conf= list(config_vec)[:(n_samp - 1)]
        new_conf[0]= new_conf[0] - 1
        
        prob_next= Pin * prob_left
        
        Ewens_recurs(new_conf,theta,prob_array,prob_next)
    
    
    if sum(config_vec[1:]) > 0:
        ## coalesc
        prob_right_I = prob_coal(theta,n_samp)
        
        jsel= [x for x in range(1,len(config_vec)) if config_vec[x] > 0]
        
        for sub in jsel:
            ##  coalesce for all classes still holding more than one allele.
            
            jprop= sub * (config_vec[sub - 1] + 1) / (n_samp - 1)
            
            new_conf= list(config_vec)
            new_conf[sub] -= 1
            new_conf[sub - 1] += 1
            new_conf= new_conf[:(n_samp - 1)]
            
            prob_right= prob_right_I * jprop

            prob_next= Pin * prob_right

            Ewens_recurs(new_conf,theta,prob_array,prob_next)
    
    return prob_array


    ii. Exact formula.

Ewens sampling formula, the exact solution to recursion above. Equation `2.19` of GGVE (see Index).

**verified** the output of this implementation was verified against table 2.1 (p. 48).

In [5]:
import math

def Ewens_exact(config_data,theta):
    
    n_samp= sum([(x + 1) * config_data[x] for x in range(len(config_data))])
    
    ThetaN= [theta + x for x in range(len(config_data))]
    ThetaN0= 1
    for y in ThetaN:
        ThetaN0 = ThetaN0 * y
    
    factor_front= math.factorial(len(config_data)) / ThetaN0
    
    classes= 1
    
    for j in range(len(config_data)):
        comb= theta / (j+1)
        comb= comb**config_data[j]
        
        prod= comb * (1 / math.factorial(config_data[j]))
        
        classes= classes * prod
    
    return factor_front * classes

####
config_trial = [2,0,0,0,0,1,0,0]

theta= 1

Ewens_exact(config_trial,theta)

0.08333333333333333

In [6]:
from structure_tools.Coalesce_plots import plot_Ewens

from plotly import tools

range_theta= np.linspace(.1,5,50)

config_complex= [
    [2,0,0,0,0,1,0,0],
    [4,2,0,0,0,0,0,0],
    [0,0,1,0,1,0,0,0],
    [1,1,0]
]

plot_Ewens(config_complex,range_theta)

['AC: 20000100', 'AC: 42000000', 'AC: 00101000', 'AC: 110']
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



### An attempt at an Infinite-sites algorithm.

#### co-factor.

Update on the `get_config` function to extract as well the set of observations in the numpy array (read haplotypes).


In [7]:

def get_config(dataw,nsamp):
    hap_str= [''.join([str(c) for c in x]).strip() for x in dataw]
    hap_str= {z:[x for x in range(nsamp) if hap_str[x] == z] for z in set(hap_str)}
    
    class_len= np.array([len(hap_str[z]) for z in hap_str.keys()])
    
    config= [sum(class_len == x) for x in range(1,nsamp + 1)]
    return config, hap_str



#### Application

The infinite sites model is difficult to use to estimate the probability of the data. This is because the number of possible states that can give rise to a known configuration rises very quickly as we travel back in generations. This number is increased when taking into account mutation position and sequence label. 

From GGVE:
    
   _If the model had been a sequence 1000 bp long with four nucleotides, the full history would have had 4^1000 ≈ 10^600 states_

Since the limitation is in computation time and memory, there have been attempts to accelerate / lighten this task through clever use of data structures and algorithms. My own attempt here is somewhat inspired by the dynamic algorithm proposed by Wu (2010).

My idea rests on a partition of previous generations / steps into layers. A haplotype data set is stored at each layer and ancestral states are stored only as the index and number of the haplotypes they represent. The `hap` set is updated with every new haplotype produced by the removal of a mutation. A dictionary of pointers is created to store information on how to travel along the ancestry tree created. 


In [8]:
import time
from sklearn.metrics import pairwise_distances


def Inf_sites(Dict_mat,point_up,layer_range= 10, print_layer= True,print_time= True,sub_sample= 0, poppit= False):
    
    t1 = time.time()
   
    MRCA= False
    
    layer= 0
    
    
    for layer in range(layer_range):
        
        if MRCA:
            continue
        
        if print_layer:
            print('layer: {}; len: {}'.format(layer,len(Dict_mat[layer])-1))
        
        if len(Dict_mat[layer]) == 2:
            stdlone= max(Dict_mat[layer].keys())
            if sum(Dict_mat[layer][stdlone][:,1]) == 1:
                MRCA = True
                continue

        if poppit:
            if layer > 1:
                Dict_mat.pop(layer - 1)
            
        hap= list(Dict_mat[layer][-2])
        hap_coord= {}
        
        point_up[layer]= []
        
        Dict_mat[layer + 1]= {   
        }
        
        Quasi= []
        nodes= []
        new_haps= []
        
        keys_get= list(Dict_mat[layer].keys())
        
        if sub_sample:
            keys_get= np.random.choice(keys_get,sub_sample)
        
        for desc in keys_get:
            
            if desc >= 0:
                
                packet= list(Dict_mat[layer][desc])
                packet= np.array(packet)

                pack_above= [x for x in range(packet.shape[0]) if packet[x,1] > 1]
                pack_below= [x for x in range(packet.shape[0]) if packet[x,1] == 1]
                
                new_entries= np.array(list(range(len(pack_above)))) + len(Dict_mat[layer + 1])
                
                for nn in range(len(new_entries)):
                    tn= new_entries[nn]
                    while tn in nodes: 
                        tn += 1
                    
                    new_entries[nn]= tn
                
                who_loses= []
                
                ### Coalescence
                for z in range(len(pack_above)):
                    
                    who_loses.append(packet[pack_above[z],0])
                    
                    pack_synth= list(packet)
                    pack_synth= np.array(pack_synth)

                    pack_synth[pack_above[z],1] -= 1
                    
                    pack_tuple= sorted([tuple(x) for x in pack_synth])
                    
                    Query= [pack_tuple == x for x in Quasi]
                    Query= np.array(Query,dtype= int)
                    Query= np.where(Query == 1)[0] ## check if this changes anything
                    
                    if len(Query):
                        new_entries[z] = nodes[Query[0]]
                        
                    else:
                        pack_synth= np.array([list(x) for x in pack_tuple])
                        
                        pack_synth= pack_synth[pack_synth[:,1] > 0]
                        Dict_mat[layer + 1][new_entries[z]]= pack_synth
                        Quasi.append(pack_tuple)
                        nodes.append(new_entries[z])
                                
                packet_mob= packet[pack_above,:]
                
                packet_mob[:,1]= 1
                packet_mob[:,0]= who_loses
                packet_mob= np.hstack((np.zeros((packet_mob.shape[0], 1), dtype=packet_mob.dtype),packet_mob))
                packet_mob= np.hstack((packet_mob,np.zeros((packet_mob.shape[0], 1), dtype=packet_mob.dtype)))
                packet_mob[:,3] = -1 #######
                packet_mob[:,0]= new_entries
                packet_mob= np.hstack((np.zeros((packet_mob.shape[0], 1), dtype=packet_mob.dtype),packet_mob))
                packet_mob[:,0]= desc
                
                for y in packet_mob:
                    point_up[layer].append(y)
                                
                ## muts that can be removed. Assume mutations happen only once.
                exist= np.array(packet)[:,0]
                exist= np.array(hap)[exist,:]
                single= np.sum(exist,axis= 0)
                single= np.where(single==1)[0]
                ##
                    
                for edan in pack_below:
                    #
                    seq= hap[packet[edan,0]]
                    
                    #print(seq)
                    who= np.where(seq == 1)[0]
                    
                    who= [x for x in who if x in single]
                    
                    if len(who) == 0:
                        continue
                    
                                        
                    for mut in who:
                        
                        tribia= list(seq)
                        tribia= np.array(tribia)
                        tribia[mut]= 0

                        calc= pairwise_distances(np.array(tribia).reshape(1,-1), hap,
                                                        metric='hamming')[0]
                        
                        match= [x for x in range(len(calc)) if calc[x] == 0] 
                        
                        if len(match):
                            #print(match)
                                                        
                            for cl in match:
                                
                                pack_synth= list(Dict_mat[layer][desc])
                                pack_synth= np.array(pack_synth)
                                pack_synth[edan,1] -= 1
                                pack_synth= pack_synth[pack_synth[:,1] > 0]
                                
                                if cl in pack_synth[:,0]:
                                    cl_idx= list(pack_synth[:,0]).index(cl)
                                    pack_synth[cl_idx,1] += 1
                                    
                                else:
                                    new_row= np.array([cl,1])
                                    pack_synth= np.vstack((pack_synth,new_row.reshape(1,-1)))
                                
                                #### make function Query existant
                                new_entry= len(Dict_mat[layer + 1])
                                
                                while new_entry in Dict_mat[layer + 1].keys():
                                    new_entry += 1
                                
                                ###
                                path_find= 0 #########
                                pack_tuple= sorted([tuple(x) for x in pack_synth])

                                Query= [pack_tuple == x for x in Quasi]
                                Query= np.array(Query,dtype= int)
                                Query= np.where(Query == 1)[0] ## check if this changes anything

                                if len(Query):
                                    new_entry= nodes[Query[0]]

                                else:
                                    #print(pack_synth)
                                    pack_synth= np.array([list(x) for x in pack_tuple])
                                    Dict_mat[layer + 1][new_entry]= pack_synth
                                    Quasi.append(pack_tuple)
                                    nodes.append(new_entry)
                                ### 

                                point_up[layer].append([desc,new_entry,cl,path_find,mut]) ############
                        
                        else:
                            #
                            if len(new_haps):
                                #
                                calc= pairwise_distances(np.array(tribia).reshape(1,-1), np.array(new_haps),
                                                                                        metric='hamming')[0]
                                
                                match= [x for x in range(len(calc)) if calc[x] == 0]
                                
                                if len(match):
                                    
                                    new_idx= len(hap) + match[0]
                                
                                else:
                                    new_haps.append(tribia)
                                    new_idx= len(hap) + len(new_haps) - 1
                            
                            else:
                                new_haps.append(tribia)
                                new_idx= len(hap)
                            
                            #
                            pack_synth= list(Dict_mat[layer][desc])
                            pack_synth.append([new_idx,1]) # pack_synth.append([len(pack_synth),1])
                            pack_synth= np.array(pack_synth)
                            pack_synth[edan,1] -= 1
                            pack_synth= pack_synth[pack_synth[:,1] > 0]
                            
                            #### make function Query existant
                            new_entry= len(Dict_mat[layer + 1])
                            while new_entry in Dict_mat[layer + 1].keys():
                                new_entry += 1
                            
                            ###
                            path_find= 0 #########
                            pack_tuple= sorted([tuple(x) for x in pack_synth])

                            Query= [pack_tuple == x for x in Quasi]
                            Query= np.array(Query,dtype= int)
                            Query= np.where(Query == 1)[0] ## check if this changes anything

                            if len(Query):
                                new_entry = nodes[Query[0]]

                            else:
                                
                                pack_synth= np.array([list(x) for x in pack_tuple])
                                Dict_mat[layer + 1][new_entry]= pack_synth
                                Quasi.append(pack_tuple)
                                nodes.append(new_entry)

                            ####
                            point_up[layer].append([desc,new_entry,new_idx,path_find,mut])
        
        if new_haps:
            
            hap.extend(new_haps)
        
        point_up[layer]= np.array(point_up[layer])
        Dict_mat[layer + 1][-2] = np.array(hap)
        
        layer += 1
    
    t2 = time.time()
    tscale= 's'
    tpass= t2 - t1
    
    if tpass > 600:
        tpass = tpass / 60
        tscale= 'm'
    
    tpass= round(tpass,3)
    if print_time:
    
        print('time elapsed: {} {}'.format(tpass,tscale))
    
    return Dict_mat, point_up



### testing

Prepare a data set and dictionaries that feed into the algorithm.  Below, the first dataset presented is meant to emulate the Ancestral Configuration `a(1,1,0)` already seen in the infinite alleles section. The second data set was taken from figure 2.10 (GGVE, pp. 52).

In [9]:
###Generate data from config

dataT= [
    [1,0,0,0],
    [0,0,1,0],
    [0,0,1,0]
]

### example from figure 2.10.

dataT= [
    [1,1,0,0],
    [1,1,0,1],
    [0,0,0,0],
    [0,0,1,0],
    [0,0,1,0]
]

dataT= np.array(dataT)

nsamp= dataT.shape[0]

config_dataw, hap_str= get_config(dataT,nsamp)

hap_sol= list(hap_str.keys())
hap_sun= np.array([np.array(list(x),dtype= int) for x in hap_sol])

hap_size= [len(hap_str[x]) for x in hap_sol]
hap_size= {z:[x for x in range(len(hap_size)) if hap_size[x] == z] for z in list(set(hap_size))}



passing= hap_size.keys()
pack= list(it.chain(*[hap_size[x] for x in passing]))
passport= list(it.chain(*[[x]*len(hap_size[x]) for x in passing]))

pack= [[pack[x],passport[x]] for x in range(len(pack))]
pack= sorted(pack)
pack= np.array(pack)

Dict_mat= {0: 
           {
               -2: hap_sun,
               -1: [0] * hap_sun.shape[0],
               0: pack
              }
          }

point_up= recursively_default_dict()


In [10]:
root_lib, point_up = Inf_sites(Dict_mat,point_up,layer_range= 10,sub_sample= 0,poppit= False)


layer: 0; len: 2
layer: 1; len: 2
layer: 2; len: 3
layer: 3; len: 5
layer: 4; len: 5
layer: 5; len: 5
layer: 6; len: 4
layer: 7; len: 1
layer: 8; len: 1
time elapsed: 0.018 s


Algorithm appears successful. Number of layers and number of ACs per layer correspond to those in `Figure 2.10` (GGVE,p. 52)

_a lesson learned on impossible ancestral states_ 

Accounted for here by eliminating only those mutations carried by a unique singleton. 

#### Traversing the Tree.

The set of AC connections produced by the algorithm above is independent of coalescent and mutation probabilities. Simply, it holds (if it is successful, which cannot be said at the present, but anticipating a breakthrough), all possible states and connections given the observed data. 

In fact, to calculate the probability of this data, we do not need to revisit the ACs created. We will use the library of pointers, which holds the nodes at each layer, the edges that connect them, and a binary marker indicating wether the link represents a mutation or coalescence envent. 

#### 2 versions of this algorithm. 

Below i have attempted two approaches at tackling the problem of traversing the tree. The first is typical:

    - Start at source, recurse on possible events going up the tree (backwardsd in time) until the sink is encountered.
    
The second approach is based on the following idea:
    
    - Make tree Acyclical before considering weights.

Coalescence paths can quickly become very numerous. However, one observation we can make is that certain paths will converge, i.e., cross one node (read _ancestral state_), diverge, then further along the path cross the another node. This can happen quite often and in close proximity. Consider the following:

`
                            3.  (MRCA)              ___________

                                                         | 
                                                    ___________
                            2.                      ___________

                                                    /        \
                                                   /          \
                                      __x________               ___________
                            1.        ___________               _______x___
                                                  \            /
                                                   \          /

                                                    __x________
                            O.                      _______x___


`

If we identify the two paths that converge, if we know where they begin, where they end and especially, how much this costs, then we can reduce the passage from layer `0` to layer `2.` from two steps into a single step. 

It first isolates all "unique" paths, basically splitting them as soon as two paths converge. Then the recursion is performed forward in time from the sink (the globam MRCA). Unfortunately it is not enough to make up for the time lost in identifying and ordering convergent nodes.

I wouldn't say this has been a waste of my time. But it doesn't feel great either..

In [11]:
#### Getting probab

### wasn't taking into account the probability of each given the population size.

def runUp_balance(Up_lib,Root,layer= 0,start= 0,Theta= 1,probs= [],prob_vec= [],Pin= 1):
    #print(layer)
    
    if not len(Up_lib[layer]):
        #print('hi')
        prob_vec.append(Pin)
        
        return prob_vec
    
    Ways_all= Up_lib[layer]
    Ways= Ways_all[Ways_all[:,0] == start]
    #print(layer)
    
    
    for row in range(Ways.shape[0]):
        action= Ways[row,3]

        ## identifying the next node.
        next_stop= Ways[row,1]
        node_next= Root[layer + 1][next_stop]

        ## calculate mut. and coal. probs based on next node sample size. 
        this_mat= Root[layer][start]
        nsamp= sum(this_mat[:,1])

        mut_prob= prob_mut(Theta,nsamp)
        coal_prob= prob_coal(Theta,nsamp)

        ### Mut was coded to 0, coalescence to 1.
        probs= [mut_prob,coal_prob]

        probe= probs[action] # edge = [mutation, coalescence] 

        ###

        who_lost= Ways[row,2] # hap set that originates the mutation / coalescent event
        hap_split= node_next[node_next[:,0] == who_lost] # hap set row

        if action == 1:
            # coalescence 
            prob_split= (hap_split[0,1]) / sum(node_next[:,1]) # proportion of ancestral hap set in previous AC

            probe= probe * prob_split

        if action == 0:
            
            # mutation
            this_mat= Root[layer][start]
            singletons= this_mat[this_mat[:,1] == 1].shape[0]
            prob_split= singletons / this_mat.shape[0]

            #prob_split= (hap_split[0,1]) / (sum(node_next[:,1])) # probability that this particular hap mutated.

            probe= probe * prob_split 


        ###

        new_pin= Pin * probe ## Probability inheritance.
        
        runUp_balance(Up_lib,Root,layer= layer + 1,start=next_stop,
                      Theta= Theta,probs= probs,prob_vec= prob_vec,Pin= new_pin)
    
    return prob_vec



The above code does indeed become intractable as the data set grows. 
Even if the number of states across layers never rises much the number of possible paths quickly becomes very large.
Here i propose another aproach:
to not repeat paths. 

the first recursive pass identifies path convergence and breaks. Paths are stored along with value in them.
second pass backwards (or forwards in time), connects all branches.


In [12]:


def runUp_unique(Up_lib,Root,layer=0,start= 0,ori= [],store_unique= {},Theta= 1,probs= [],prob_vec= [],Pin= 1):
    
    if not len(Up_lib[layer]):
        
        if ori not in store_unique.keys():
            
            end_story= (layer,start)
            
            store_unique[ori][end_story]= Pin
            

        return store_unique
    
    Ways_all= list(Up_lib[layer])
    Ways_all= np.array(Ways_all)
    Ways= Ways_all[Ways_all[:,0] == start]    
    
    for row in range(Ways.shape[0]):
        ori_here= tuple(ori)
        action= Ways[row,3]

        ## identifying the next node.
        next_stop= Ways[row,1]
        node_next= Root[layer + 1][next_stop]
        
        if len(ori) < 3:
            ori_here= (ori[0],ori[1],next_stop)
            
        #if ori_here in store_unique.keys():
        #    return store_unique

        ## calculate mut. and coal. probs based on next node sample size. 
        this_mat= Root[layer][start]
        nsamp= sum(this_mat[:,1])

        mut_prob= prob_mut(Theta,nsamp)
        coal_prob= prob_coal(Theta,nsamp)

        ### Mut was coded to 0, coalescence to 1.
        probs= [mut_prob,coal_prob]
        
        probe= probs[action] # edge = [mutation, coalescence] 

        ###

        who_lost= Ways[row,2] # hap set that originates the mutation / coalescent event
        hap_split= node_next[node_next[:,0] == who_lost] # hap set row
        

        if action == 1:
            # coalescence 
            prob_split= (hap_split[0,1]) / sum(node_next[:,1]) # proportion of ancestral hap set in previous AC

            probe= probe * prob_split

        if action == 0:
            # mutation
            
            singletons= this_mat[this_mat[:,1] == 1].shape[0]
            prob_split= singletons / this_mat.shape[0]
            #prob_split= (hap_split[0,1]) / sum(node_next[:,1])  # probability that this particular hap mutated.

            probe= probe * prob_split 
        
        ###

        new_pin= Pin * probe ## Probability inheritance.
        
        if Ways_all[Ways_all[:,1] == next_stop].shape[0] > 1:
            next_story= (layer + 1, next_stop)
            
            store_unique[ori_here][next_story]= new_pin
            
            runUp_unique(Up_lib,Root,layer=layer + 1,start=next_stop,ori=next_story,store_unique=store_unique,
                 Theta= Theta,probs= probs,prob_vec= prob_vec,Pin= 1)
        
        else:
        
            runUp_unique(Up_lib,Root,layer + 1,start= next_stop,ori=ori_here,store_unique=store_unique,
                         Theta= Theta,probs= probs,prob_vec= prob_vec,Pin= new_pin)
    
    return store_unique




def split_browse(browse_dict,permission= True):
    
    skeys= list(browse_dict.keys())
    
    dirs= list(it.chain(*[[x]*len(browse_dict[x]) for x in skeys]))
    
    sets= list(it.chain(*[browse_dict[x] for x in skeys]))
    
    first_lib= {
        z: {dirs[x]:browse_dict[dirs[x]][sets[x]] for x in range(len(sets)) if sets[x] == z} for z in list(set(sets))
    }
    
    ### compression step 
    ### get rid of cycles
    
    if permission:
        for cl in first_lib.keys():

            new_roots= list(first_lib[cl].keys())
            new_suf= [x[:2] for x in new_roots]

            new_seeds= {
                z: [x for x in range(len(new_suf)) if new_suf[x] == z] for z in list(set(new_suf))
            }
            
            new_seeds= {
                new_roots[new_seeds[z][0]]: sum([first_lib[cl][new_roots[x]] for x in new_seeds[z]]) for z in new_seeds.keys()
            }

            first_lib[cl]= new_seeds
    
    ###
    ###
        
    return first_lib



def rec_back(node,node_input,val_vec= [],Pin= 1):
    
    if node[:2] == (0,0):
        
        val_vec.append(Pin)
        return val_vec
    
    if node not in node_input.keys():
        print('node {} has no descendents.'.format(node))

    for desc in node_input[node].keys():
        
        new_desc= tuple(desc[:2])
        new_pin= Pin * node_input[node][desc]
        
        rec_back(new_desc,node_input,val_vec= val_vec,Pin=new_pin)

    return val_vec



def run_unique_combine(Up_lib= {},Root= {},layer= 0,start= 0,origin= [0,0],store_unique= {},
                       Theta= 1,probs= [],prob_vec= [],Pin= 1):
    
    store_unique= recursively_default_dict()
    
    Browse= runUp_unique(Up_lib= Up_lib,Root= Root,layer= layer,start= start,ori= origin,
                         store_unique= store_unique,Theta= Theta,probs= probs,prob_vec= prob_vec,Pin= Pin)
    
    Uni_ends = split_browse(Browse)
    sink= max(Uni_ends.keys())
    
    paths_forward= rec_back(sink,Uni_ends,val_vec= [],Pin= 1)
    
    return paths_forward




In [13]:
Theta= 1
ori= (0,0)
prob_vec= []

start= 0
layer= 0
origin= (layer,start)

store_unique= recursively_default_dict()

Browse= runUp_unique(Up_lib= point_up,Root= root_lib,layer= layer,start= start,ori= origin,
                     store_unique= store_unique,Theta= Theta,prob_vec= prob_vec)

Uni_ends = split_browse(Browse)
sink= max(Uni_ends.keys())

paths_forward= rec_back(sink,Uni_ends,val_vec= [],Pin= 1)

### Theta

Probability of the data under varying values of _Theta_.

In [14]:
from structure_tools.Coalesce_plots import plot_rec_InfSites

from plotly import tools

func_names= ['run_up','run_up_split']
funcs= [runUp_balance,
        run_unique_combine
       ]

range_theta= np.linspace(.1,10,100)

plot_rec_InfSites(point_up,root_lib,funcs,func_names,range_theta,height= 900)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



According to Figure 2.16 of GGVE, maximum values of theta are around 2.12, with probabilities under 7e-4. It is looking quite close for both approaches. But we can see that we actually lose time with the split recursive. The time it takes to reorganize every path clearly does not make up for the small gain in time it represents when backtracking..


## Back to the drawing board.

Recurrent implementations are attractive because they are also a bit lazy. It does not help that they are hard to debug too. Let's revisit a dynamic implementation.

### Faisal _et al._ 2015.

Dynamic algorithm. More straighforward just more complicated to code too.

- To construct the tree is actually quite fast (see above). I was feeling frustrated to be forced to travel all possible paths to calculate the quantities necessary. But as Faisal _et al._ point out, it isn't necessary with the correct indexing. 

Everything needed is already in place. The algorithm that constructs the tree keeps track of the edges that connect every two nodes between two subsequent layers. One only has to reuse this dictionary (here, the `point_up` dict) to coordinate the summation of path probabilities. Sample size calculations performed by referring to the `root_lib` dictionary, which stores information on all ACs, saving space by retainning only haplotype index and copy number information in two-column numpy arrays.

In [15]:

def tree_descent(root_lib,point_up,sink,init= [0],Theta= 1):

    edge_weights= recursively_default_dict()
    paths= recursively_default_dict()
    paths_where= recursively_default_dict()
    
    for layer in list(range(1,sink + 1))[::-1]:
        
        #print('layer: {}'.format(layer))
        where_to= point_up[layer - 1]
        
        if layer == sink:
            starters= init
        else:
            starters= list(set(where_to[:,1]))
        #print(where_to)
        
        for desc in list(set(where_to[:,1])):
            
            ###
            #print(packet)
            ###
            
            point_up_house= where_to[where_to[:,1] == desc]
    
            #point_up[layer].extend(point_up_house)
            #point_up_house= np.array(point_up_house)

            if not len(paths):

                paths= {
                    0: 1
                }

                paths_where[layer][desc]= [1]


            for row in range(point_up_house.shape[0]):
                pack= list(paths_where[layer][desc])

                here= desc #point_up_house[row,1]
                node_next= np.array(root_lib[layer][here])

                who_lost= point_up_house[row,2] # hap set that originates the mutation / coalescent event
                hap_split= node_next[node_next[:,0] == who_lost] # hap set row
                
                if row > 0:
                    new_entry= len(paths)
                    while new_entry in paths.keys():
                        new_entry += 1
                
                going= point_up_house[row,0]
                
                ###
                packet= list(root_lib[layer-1][going])
                packet= np.array(packet)
                nsamp= sum(packet[:,1]) ## Samp next AC
                
                mut_prob= prob_mut(Theta,nsamp)
                coal_prob= prob_coal(Theta,nsamp)

                prob_vec= [mut_prob,coal_prob]
                
                ### mut proportion
                reff= root_lib[layer-1][going]
                reff_sing= reff[reff[:,1] == 1]
                
                mut_prop= reff_sing.shape[0] / reff.shape[0]
                
                ### This makes the difference.              
                nsampn= sum(root_lib[layer][desc][:,1]) ### Samp this AC
                
                ####
                if point_up_house[row,3]== 1:
                    prob_split= (hap_split[0,1]) / nsampn
                
                else:
                    prob_split= mut_prop
                
                pack= [x * prob_vec[point_up_house[row,3]] * prob_split for x in pack]

                if going not in paths_where[layer - 1].keys():
                    paths_where[layer - 1][going]= pack

                else:
                    paths_where[layer - 1][going].extend(pack)



        if len(edge_weights) == 0:
            edge_weights[sink][0] = 1

        for desc in paths_where[layer - 1].keys():
            edge_weights[layer - 1][desc]= sum(paths_where[layer - 1][desc])
            paths_where[layer - 1][desc]= [sum(paths_where[layer - 1][desc])]
            

    return edge_weights, paths_where



def Descent_return(point_up,root_lib,layer=0,start=0,Theta= 1,prob_vec= []):
    
    
    sink= max(root_lib.keys())
    
    if 0 not in root_lib[sink].keys():
        while 0 not in root_lib[sink].keys():
            sink -= 1
    
    
    
    node_weigths, paths_reverse = tree_descent(root_lib,point_up,sink,Theta= Theta)
       
    #return paths_reverse
    return [node_weigths[0][0]]



In [16]:
### Another take, backwards instead of forward in time.
###

def tree_ascent(root_lib,point_up,sink,init= [0],Theta= 1):
    
    edge_weights= recursively_default_dict()
    paths= recursively_default_dict()
    paths_where= recursively_default_dict()
    
    layer_nodes= {}
    
    for layer in list(range(sink)):
        
        where_to= point_up[layer]
        
        theta_now= [Theta]
        layer_nodes[layer]= []
        
        for desc in list(set(where_to[:,0])):
            
            ###
            #print(packet)
            ###
            
            point_up_house= where_to[where_to[:,0] == desc]
    
            if not len(paths):

                paths= {
                    0: 1
                }
                
                paths_where[layer][desc]= [1]
            
            for row in range(point_up_house.shape[0]):
                pack= list(paths_where[layer][desc])
    
                there= point_up_house[row,1]
                node_next= list(root_lib[layer + 1][there])
                node_next= np.array(node_next)
                
                who_lost= point_up_house[row,2] # hap set that originates the mutation / coalescent event
                hap_split= node_next[node_next[:,0] == who_lost] # hap set row
                #print(hap_split)
                
                if row > 0:
                    new_entry= len(paths)
                    while new_entry in paths.keys():
                        new_entry += 1
                
                going= point_up_house[row,1]
                
                ###
                packet= list(root_lib[layer+1][going])
                packet= np.array(packet)
                
                ###
                reff= list(root_lib[layer][desc])
                reff= np.array(reff)
                nsampn= sum(reff[:,1]) ### Samp this AC
                nsamp= sum(packet[:,1]) ## Samp next AC
                
                mut_prob= prob_mut(Theta,nsampn)
                coal_prob= prob_coal(Theta,nsampn)

                prob_vec= [mut_prob,coal_prob]
                
                ### mut proportion
                
                reff_sing= reff[reff[:,1] == 1]
                
                mut_prop= reff_sing.shape[0] / reff.shape[0]
                
                ####
                if point_up_house[row,3]== 1:
                    prob_split= (hap_split[0,1]) / nsamp
                
                else:
                    prob_split= mut_prop
                
                ####
                ####
                pack= [x * prob_vec[point_up_house[row,3]] * prob_split for x in pack]

                if going not in paths_where[layer + 1].keys():
                    paths_where[layer + 1][going]= pack

                else:
                    paths_where[layer + 1][going].extend(pack)

        if len(edge_weights) == 0:
            edge_weights[sink][0] = 1

        for desc in paths_where[layer + 1].keys():
            edge_weights[layer + 1][desc]= sum(paths_where[layer + 1][desc])
            paths_where[layer + 1][desc]= [sum(paths_where[layer + 1][desc])]
            

    return edge_weights, paths_where



def Ascent_return(point_up,root_lib,layer=0,start=0,Theta= 1,prob_vec= []):
    
    
    sink= max(root_lib.keys())
    
    if 0 not in root_lib[sink].keys():
        while 0 not in root_lib[sink].keys():
            sink -= 1
    
    
    
    node_weigths, paths_reverse = tree_ascent(root_lib,point_up,sink,Theta= Theta)
       
    #
    return [node_weigths[sink][0]]



### Test

In [17]:
func_names= ['run_up','run_up_split']
func_names= ['run_up','run_up_split','descent_return','ascent_return']
funcs= [
        runUp_balance,
        run_unique_combine,
        Descent_return,
        Ascent_return
       ]

range_theta= np.linspace(0.1,10,100)

plot_rec_InfSites(point_up,root_lib,funcs,func_names,range_theta,height= 900)


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



The dynamic approach is much faster that either recurrent implementation. Probability values and maximum correspond to the examples in the book (2.12, GGVE p. 54, Fig. 2.13).

### Changing ancestor haps

The probability of the present data set given a subset of possible ancestral states. 

Here we consider all ancestral states where a given sequence is first observed. Paths from those ACs to the present data
are summed.

In [18]:
from structure_tools.Coal_tools import get_sinks

range_theta= np.linspace(0.1,10,100)

Anc_poss= [
    [0,0,0,0],
    [1,0,0,0],
    [0,1,0,0],
    [1,1,0,0],
    [1,1,0,1]
]

Anc_poss= np.array(Anc_poss)

mrca= Anc_poss[1]
        
get_sinks(mrca,root_lib,point_up)

(6, [1])

In [19]:
from structure_tools.Coalesce_plots import plot_InfSites_mrca

plot_InfSites_mrca(Anc_poss,point_up,root_lib,range_theta,height= 500,width= 900)

### Quantities.

### I. Number of haplotypes.


### I. Number of segregating sites.


In [261]:
from math import factorial
from operator import mul    # or mul=lambda x,y:x*y
from fractions import Fraction
from functools import reduce

def nCk(n,k):
  return int( reduce(mul, (Fraction(n-i, i+1) for i in range(k)), 1) )


def rec_sSN(k,Nsamp,pi,Theta= 1,held= [], init= 1):
    #print(sum(held))
    init_t= int(init==1)
    #print(init_t)
    
    init_c= [2,1][init_t]
    
    
    if Nsamp == init_c:
        if init_c== 1 and k == 0:
            #print(init)
            um= pi * init
            held.append(um)
        
        if init_c == 2:
            um= pi * Ps2(k,2)
            held.append(um)
            
        
    
    else:
        ##
        mut= Theta / (Nsamp - 1 + Theta)
        sub= (Nsamp-1) / (Nsamp-1+Theta)

        mut_d= pi * mut
        coal_d= pi * sub
        
        if k > 0:
            
            rec_sSN(k-1,Nsamp,mut_d,Theta= Theta,held= held,init= init)
            
        rec_sSN(k,Nsamp- 1,coal_d,Theta= Theta,held= held,init=init)
    
    return held



def Ps2(k,Theta):
    
    su= (Theta / (Theta + 1))**k / (Theta + 1)
    
    return su


def exact_SN(k,n,Theta):
    
    if k==0:
        dis= [Theta + x for x in range(n)]
        dis= reduce((lambda x, y: x * y), dis)
        
        su= factorial(n - 1) / dis
        
        return su
    
    preff= (n-1) / Theta
    weight= 0
    
    for i in range(1,n):
        
        mu= (-1)**(i-1)
        mu= mu * nCk(n-2,i-1)
        
        mu= mu * (Theta / (i + Theta))**(k+1)
        
        weight += mu
    
    Sk= preff * weight
    
    return Sk


def Sn_stats(n,Theta):
    
    mean= Theta * sum([1/x for x in range(n)])
    var= mean + Theta**2 * sum([1/(x**2) for x in range(n)])
    Exp= {
        'mean': mean,
        'var': var,
        'sing': Theta * (1 + 1 / (n - 1))
    }



In [262]:
Nsamp= 7
Theta= 1.8

X_range= [0,15]
X_plot= list(range(X_range[0],X_range[1]))

##
fig_k= []

##
Tk= []

for k in X_plot:
    pi= 1
    held= []
    
    um= rec_sSN(k,Nsamp,pi,Theta= Theta,held= held, init= 1) #
    um= sum(um)
    
    Tk.append(um)
    
trace1= go.Scatter(
    x= X_plot,
    y= Tk,
    mode= 'lines',
    name= 'recInit 1'
)

fig_k.append(trace1)

##
Tk= []

for k in X_plot:
    pi= 1
    held= []
    
    um= rec_sSN(k,Nsamp,pi,Theta= Theta,held= held, init= 2) #
    um= sum(um)
    
    Tk.append(um)
    
trace1= go.Scatter(
    x= X_plot,
    y= Tk,
    mode= 'lines',
    name= 'recInit S2'
)

fig_k.append(trace1)


##
Tk= [Ps2(k,Theta) for k in X_plot]


trace1= go.Scatter(
    x= X_plot,
    y= Tk,
    mode= 'lines',
    name= 'S2'
)

fig_k.append(trace1)
###

Tk= [exact_SN(k,Nsamp,Theta) for k in X_plot]

trace1= go.Scatter(
    x= X_plot,
    y= Tk,
    mode= 'lines',
    name= 'exact'
)

fig_k.append(trace1)


layout= go.Layout(
    title= 'Prob of Sk n= {}, theta= {}'.format(Nsamp,Theta),
    yaxis= dict(title= 'P'), #,range= [0,1]),
    xaxis= dict(title= 'k')
)


Fig= go.Figure(data= fig_k, layout= layout)
iplot(Fig)